# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# data processing libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re

#nlp text processing libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('wordnet')

#ml processing libraries
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

import pickle

import warnings
warnings.simplefilter("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_messages.db')
df = pd.read_sql_table("disaster_messages.db",  engine)

# trim df to only have rows with wanted labels/tags i,e related column should only have 0 OR 1
df = df[(df["related"] == 0)|(df["related"] == 1)]

X = df["message"]
Y = df.drop(["id", "message", "original", "genre"], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    function to normalize and tokenize text i.e disaster messages received
    
    Inputs:
    Pandas series containing disaster messages
    
    
    Outputs:
    List of words that have been processed through provided arguments
    """
    
    # converting all text to lowercase
    text = re.sub(r"[^a-zA-Z0-9]", " ", str(text).lower())
    
    
    # tokenize words
    tokens = word_tokenize(text)
    
    
    #lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    
    # normalize word tokens and remove stop words
    normalizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normalized = [normalizer.stem(word) for word in clean_tokens if word not in stop_words]
    
    return normalized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ("vect", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(RandomForestClassifier())) 
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)

np.random.seed(0)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def evaluate_metrics(y_train_array, y_pred_array, col_names):
    """Evalute metrics of the ML pipeline model by iterating 
    
    Inputs:
    y_train_array - array containing target variable real labels.
    y_pred_array - array containing target variable predicted labels.
    col_names - list of strings containing names for each of the y_pred_array fields.
       
    Outputs:
    data_metrics - dataframe containing accuracy, precision, recall and f1 score for a given set of y_train_array and y_pred_array labels.
    """
    metrics = []
    
    #evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(y_train_array[:, i], y_pred_array[:, i])
        precision = precision_score(y_train_array[:, i], y_pred_array[:, i])
        recall = recall_score(y_train_array[:, i], y_pred_array[:, i])
        f1 = f1_score(y_train_array[:, i], y_pred_array[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    #sav metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ["Accuracy", "Precision", "Recall", "F1"])
      
    return data_metrics

In [7]:
#calculate evaluation metrics for training set
Y_train_pred = pipeline.predict(X_train)
col_names = list(Y.columns.values)

print(evaluate_metrics(np.array(Y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.990113   0.992599  0.994522  0.993559
request                 0.988576   0.996529  0.937092  0.965897
offer                   0.998771   1.000000  0.739130  0.850000
aid_related             0.984478   0.994078  0.968574  0.981160
medical_help            0.988269   0.996221  0.854734  0.920070
medical_products        0.991445   0.995043  0.831263  0.905809
search_and_rescue       0.993289   1.000000  0.753759  0.859593
security                0.995441   1.000000  0.736686  0.848382
military                0.995236   0.996337  0.856693  0.921253
child_alone             1.000000   0.000000  0.000000  0.000000
water                   0.993392   1.000000  0.896135  0.945223
food                    0.994570   0.999043  0.952555  0.975245
shelter                 0.992879   0.998105  0.920746  0.957866
clothing                0.998105   1.000000  0.878289  0.935201
money                   0.994980   1.000

In [8]:
#calculate evaluation metrics for test set
Y_test_pred = pipeline.predict(X_test)

eval_metrics_1 = evaluate_metrics(np.array(Y_test), Y_test_pred, col_names)
print(eval_metrics_1)

                        Accuracy  Precision    Recall        F1
related                 0.807284   0.840607  0.920616  0.878794
request                 0.880436   0.753894  0.438406  0.554410
offer                   0.996004   0.000000  0.000000  0.000000
aid_related             0.753035   0.756606  0.601326  0.670088
medical_help            0.920086   0.671875  0.079336  0.141914
medical_products        0.949746   0.833333  0.072046  0.132626
search_and_rescue       0.971262   0.666667  0.052083  0.096618
security                0.979407   0.000000  0.000000  0.000000
military                0.965729   0.562500  0.040000  0.074689
child_alone             1.000000   0.000000  0.000000  0.000000
water                   0.948978   0.845070  0.279070  0.419580
food                    0.934378   0.839286  0.514364  0.637829
shelter                 0.932995   0.806818  0.356187  0.494200
clothing                0.986322   0.833333  0.148515  0.252101
money                   0.975411   0.785

In [9]:
#calculate the proportion of each column that have label == 1
Y.sum()/len(Y)

related                   0.764792
request                   0.171892
offer                     0.004534
aid_related               0.417243
medical_help              0.080068
medical_products          0.050446
search_and_rescue         0.027816
security                  0.018096
military                  0.033041
child_alone               0.000000
water                     0.064239
food                      0.112302
shelter                   0.088904
clothing                  0.015560
money                     0.023206
missing_people            0.011449
refugees                  0.033618
death                     0.045874
other_aid                 0.132396
infrastructure_related    0.065506
transport                 0.046143
buildings                 0.051214
electricity               0.020440
tools                     0.006109
hospitals                 0.010873
shops                     0.004610
aid_centers               0.011872
other_infrastructure      0.044222
weather_related     

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# Define performance metric for use in grid search scoring object
#def perform_metric(Y_T, Y_P):
def performance_metrics_grid_search(y_train_array, y_pred_array):
    """Median F1 score for all classifiers
    
    Inputs:
    y_train_array - array containing target variable real labels.
    y_pred_array - array containing target variable predicted labels.
        
    Outputs:
    median F1 score for all  classifiers
    """
    f1_list = []
    for i in range(np.shape(y_pred_array)[1]):
        f1 = f1_score(np.array(y_train_array)[:, i], y_pred_array[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [11]:
parameters = {"vect__min_df": [1, 5],
              "tfidf__use_idf":[True, False],
              "clf__estimator__n_estimators":[10, 25], 
              "clf__estimator__min_samples_split":[2, 5, 10]}



scorer = make_scorer(performance_metrics_grid_search)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 5)

#find the best parameters
np.random.seed(25)
tuned_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.0940154157876047, total=  57.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.11981408731256571, total=  56.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.11801242236024845, total=  57.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.8min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.12962574193179, total=  49.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  5.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.14351200493616456, total=  49.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.15662104811406227, total=  49.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.10258607652366791, total=  57.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.11204709225616563, total=  57.5s
[CV] clf__estimator__min_samples_split=2, cl

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 106.1min finished


In [12]:
#grid search results
tuned_model.cv_results_

{'mean_fit_time': array([ 47.37091247,  40.35765052,  47.88753104,  39.58734155,
         93.27661999,  76.1574587 ,  95.09358795,  74.54618979,
         40.7350653 ,  37.02441001,  40.5435195 ,  36.20297837,
         77.46312229,  68.23160338,  77.34641186,  65.97923716,
         38.25211596,  35.72631788,  37.66573723,  34.67455188,
         71.1306599 ,  64.80612954,  69.95761315,  61.9532268 ]),
 'std_fit_time': array([ 0.29795552,  0.05069102,  0.20518094,  0.18139919,  0.26688755,
         0.11441379,  0.16457284,  0.07016877,  0.04367415,  0.11711199,
         0.21153493,  0.08985636,  0.29616246,  0.26580368,  0.18151832,
         0.10682591,  0.09031901,  0.08376947,  0.01403503,  0.12815117,
         0.05559873,  0.22587277,  0.16485241,  0.15264039]),
 'mean_score_time': array([  9.6658833 ,   9.31509686,   9.65439153,   9.27728017,
         11.96271165,  11.09012294,  11.88440736,  10.96895941,
          9.64531819,   9.32572969,   9.5638454 ,   9.29296891,
         11.9565

In [13]:
#top mean score
np.max(tuned_model.cv_results_["mean_test_score"])

0.1980295675369535

In [14]:
#parameters for top mean score
tuned_model.best_params_

{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 25,
 'tfidf__use_idf': False,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
#evaluating metrics for test set
tuned_pred_test = tuned_model.predict(X_test)

eval_metrics_2 = evaluate_metrics(np.array(Y_test), tuned_pred_test, col_names)

print(eval_metrics_2)

                        Accuracy  Precision    Recall        F1
related                 0.818657   0.835776  0.947145  0.887982
request                 0.892885   0.787836  0.504529  0.615130
offer                   0.996004   0.000000  0.000000  0.000000
aid_related             0.777778   0.736567  0.727340  0.731924
medical_help            0.918549   0.558824  0.105166  0.177019
medical_products        0.951898   0.774194  0.138329  0.234719
search_and_rescue       0.970954   0.588235  0.052083  0.095694
security                0.979253   0.000000  0.000000  0.000000
military                0.967727   0.758621  0.097778  0.173228
child_alone             1.000000   0.000000  0.000000  0.000000
water                   0.958814   0.871560  0.441860  0.586420
food                    0.944829   0.810000  0.664843  0.730278
shelter                 0.940218   0.797721  0.468227  0.590095
clothing                0.987859   0.750000  0.326733  0.455172
money                   0.976026   0.833

In [16]:
#summary of 1st model
eval_metrics_1.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.944752,0.567653,0.192243,0.248879
std,0.056639,0.366853,0.245765,0.271783
min,0.753035,0.000000,0.000000,0.000000
25%,0.934033,0.136364,0.003580,0.006977
50%,0.958890,0.755250,0.068761,0.126753
75%,0.982634,0.835189,0.316507,0.444592
max,1.000000,1.000000,0.920616,0.878794


In [17]:
#summary of tuned model
eval_metrics_2.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.948969,0.531167,0.241639,0.294385
std,0.051442,0.364587,0.285752,0.300433
min,0.777778,0.000000,0.000000,0.000000
25%,0.943676,0.000000,0.000000,0.000000
50%,0.962348,0.743284,0.101472,0.175123
75%,0.983249,0.811706,0.448452,0.587339
max,1.000000,1.000000,0.947145,0.887982


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#using SVM instead 
pipeline2 = Pipeline([
    ("vect", CountVectorizer(tokenizer = tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(SVC()))
])

parameters2 = {"vect__min_df": [5],
              "tfidf__use_idf":[True],
              "clf__estimator__kernel": ["poly"], 
              "clf__estimator__degree": [1, 2, 3],
              "clf__estimator__C":[1, 10, 100]}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2, scoring = scorer, verbose = 5)

#find best parameters
np.random.seed(25)
tuned_model2 = cv2.fit(X_train, Y_train)

In [ ]:
#grid search results
tuned_model2.cv_results_

In [ ]:
#calculate evaluation metrics for test set
tuned_pred_test2 = tuned_model2.predict(X_test)

eval_metrics_3 = evaluate_metrics(np.array(Y_test), tuned_pred_test2, col_names)

print(eval_metrics_3)

### 9. Export your model as a pickle file

In [20]:
#pickle best model
pickle.dump(tuned_model, open("disaster_model.sav", 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.